In [23]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from typing import Tuple, List
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import AgentExecutor
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

In [35]:
import os

os.environ["OPENAI_API_KEY"] = "your key" # OpenAI API KEY (https://platform.openai.com/api-keys)
os.environ["TAVILY_API_KEY"] = "your key" # Tavily Search API KEY (https://app.tavily.com/home)

In [48]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [49]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily Search API : 효율적이고 빠른 검색 결과를 목표로 하는 LLM 및 RAG에 최적화된 검색 엔진
search = TavilySearchResults(k=3)
search.invoke("0koang velog 주소")

[{'url': 'https://alpha.velog.io/@kwon0koang/series',
  'content': 'kwon0koang님이 작성한 포스트 시리즈들을 확인해보세요. 서비스 핵심 가치를 이해하고, 지속적인 개선을 이끄는 엔지니어(를 지향함)'},
 {'url': 'https://www.youtube.com/watch?v=l4sKIoSGuEc',
  'content': "I can't believe that I saw #Disneyland #hongkong with my own eyes. #disneylandhotel #Swisskaur #vlog #travel #trip #tripvlog"},
 {'url': 'https://www.youtube.com/watch?v=IX68zHymkyo',
  'content': "#hongkong #hongkongvlog Hello EveryoneI'm back in Asia and this time I will visit Hong Kong for the first time.Have you been? 💌This is my day 1I hope you wi..."},
 {'url': 'https://blog.youtube/creator-and-artist-stories/sam-and-victor/',
  'content': 'The gentle whirrs of an espresso machine. Curtains softly blowing by an open window. The sound of rain hitting nearby clusters of trees. Aesthetic vlogging, a trend that has grown to multiple million videos since 2020, is all about romanticizing the ordinary, and Hong Kong-based couple Sam and Victor have been dabbling in it over

In [50]:
tools = [search]

llm_with_tools = llm.bind_tools(tools)